In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df = pd.read_csv('prod.csv')


In [3]:
# Categorical columns to one-hot encode
categorical_cols = ['category', 'product_subcategory', 'brand', 'product_type', 'seasonal_indicator', 'promotion_indicator', 'customer_demographics']

X=df[['category','product_title','product_subcategory','brand','selling_price','product_description','original_price','product_type','seasonal_indicator','promotion_indicator','customer_demographics','product_rating','customer_reviews_count']]
# Perform one-hot encoding
X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# Display the encoded dataframe
X_encoded


,product_title,selling_price,product_description,original_price,product_rating,customer_reviews_count,"category_Bakery, Cakes & Dairy",category_Beauty & Hygiene,category_Beverages,category_Cleaning & Household,...,product_type_Whole Spices,product_type_Wine & Rice Vinegar,product_type_Women's Deodorants,product_type_Yogurt & Shrikhand,seasonal_indicator_Spring,seasonal_indicator_Summer,seasonal_indicator_Winter,promotion_indicator_Yes,customer_demographics_Male,customer_demographics_Other
0,Garlic Oil - Vegetarian Capsule 500 mg,220.00,This Product contains Garlic Oil that is known...,220.0,4.1,7,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
1,Water Bottle - Orange,180.00,"Each product is microwave safe (without lid), ...",180.0,2.3,54,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,"Brass Angle Deep - Plain, No.2",119.00,"A perfect gift for all occasions, be it your m...",250.0,3.4,31,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
3,Cereal Flip Lid Container/Storage Jar - Assort...,149.00,Multipurpose container with an attractive desi...,176.0,3.7,49,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
4,Creme Soft Soap - For Hands & Body,162.00,Nivea Creme Soft Soap gives your skin the best...,162.0,4.4,74,0,1,0,0,...,0,0,0,0,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27548,"Wottagirl! Perfume Spray - Heaven, Classic",199.20,Layerr brings you Wottagirl Classic fragrant b...,249.0,3.9,75,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
27549,Rosemary,67.50,Puramate rosemary is enough to transform a dis...,75.0,4.0,65,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27550,Peri-Peri Sweet Potato Chips,200.00,We have taken the richness of Sweet Potatoes (...,200.0,3.8,61,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
27551,Green Tea - Pure Original,396.00,"Tetley Green Tea with its refreshing pure, ori...",495.0,4.2,7,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1


In [4]:
X_encoded.columns

Index(['product_title', 'selling_price', 'product_description',
       'original_price', 'product_rating', 'customer_reviews_count',
       'category_Bakery, Cakes & Dairy', 'category_Beauty & Hygiene',
       'category_Beverages', 'category_Cleaning & Household',
       ...
       'product_type_Whole Spices', 'product_type_Wine & Rice Vinegar',
       'product_type_Women's Deodorants', 'product_type_Yogurt & Shrikhand',
       'seasonal_indicator_Spring', 'seasonal_indicator_Summer',
       'seasonal_indicator_Winter', 'promotion_indicator_Yes',
       'customer_demographics_Male', 'customer_demographics_Other'],
      dtype='object', length=2848)

In [15]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorize product descriptions
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=100)
tfidf_matrix = tfidf_vectorizer.fit_transform(X_encoded['product_description'].astype(str))

# Fit KNN model
knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn.fit(tfidf_matrix)

def recommend_products(product_description, knn_model, df, tfidf_vectorizer):
    # Vectorize input product description
    description_vec = tfidf_vectorizer.transform([product_description])
    
    # Find k-nearest neighbors based on product description
    distances, indices = knn_model.kneighbors(description_vec.toarray())
    
    # Get recommended product indices
    recommended_indices = indices[0]
    
    # Get recommended products based on product description
    recommended_products = df.iloc[recommended_indices]
    
    return recommended_products[[ 'product_title']]

# Test recommendation
product_description = 'Hair and Wash Beautiful skin care'

# Assuming df is your original dataframe containing product details
df = X_encoded.reset_index(drop=True)  # Reset index for df to match indices in X_encoded

recommended_products = recommend_products(product_description, knn, df, tfidf_vectorizer)

print("Recommended Products based on Product Description:")
print(recommended_products)


Recommended Products based on Product Description:
                                  product_title
3548   Passion For Vanilla Shampoo & Shower Gel
17435                         Face & Beard Wash
19249    Beard Wash For Growth - Almond & Thyme
18429             Natural Multi Purpose Cleaner
17875       Purple Delight Shampoo & Shower Gel


In [16]:
import pickle

# Save TF-IDF Vectorizer
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

# Save KNN model for TF-IDF
with open('knn_tfidf.pkl', 'wb') as f:
    pickle.dump(knn_tfidf, f)
